In [1]:
### imports
import pandas as pd
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
import os
from dotenv import load_dotenv

In [2]:
### setup
load_dotenv()
client_secret_file = os.getenv("CLIENT_SECRET_FILE")

excelFilePath = "Card Mentions & Referral Links.xlsx"
excelNewLinkPageName = "Referrals"
excelLegacyVideosPageName = "Videos"

In [ ]:
### yt authentification

client_secret_file = os.getenv("CLIENT_SECRET_FILE")
scopes = ["https://www.googleapis.com/auth/youtube.force-ssl"]
flow = InstalledAppFlow.from_client_secrets_file(client_secret_file, scopes=scopes)
credentials = flow.run_local_server(port=0)
youtube = build("youtube", "v3", credentials=credentials)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=862595706076-6uhl6pp6562616vrn8eeth2fbml431h9.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A53098%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=fbmGAdv4Tj8WFzDX27HH3MpmUtquhk&access_type=offline


In [ ]:
### excel to pd

videosDf = pd.read_excel(excelFilePath, sheet_name=excelLegacyVideosPageName)
ccLinksDF = pd.read_excel(excelFilePath, sheet_name=excelNewLinkPageName)

In [ ]:
def update_video_description(video_id, old_link, new_link):
    # Get the existing video details
    video_response = youtube.videos().list(
        part="snippet",
        id=video_id
    ).execute()

    if video_response["items"]:
        # Extract the current description
        video_snippet = video_response["items"][0]["snippet"]
        current_description = video_snippet["description"]

        # Replace old link with new link in the description
        updated_description = current_description.replace(old_link, new_link)

        # Update the video description only if there's a change
        video_snippet["description"] = updated_description
        youtube.videos().update(
            part="snippet",
            body={
                "id": video_id,
                "snippet": video_snippet
            }
        ).execute()
        print(f"Updated video {video_id} by replacing the old link with the new referral link.")


In [ ]:
for index, video_row in videosDf.iterrows():
    video_id = video_row["Video ID"]  # Assuming you have a column for video IDs
    
    # Check each credit card column for True values
    for cc_name, needs_link in video_row.items():
        if needs_link:  # Only proceed if True
            # Find the old and new referral links from the referral_links_df
            referral_link_row = ccLinksDF[ccLinksDF['Credit Card Name'] == cc_name]
            
            if not referral_link_row.empty:
                old_link = referral_link_row["Old Link"].values[0]
                new_link = referral_link_row["New Link"].values[0]

                if old_link != new_link:
                    # Update the video description if the old link is found
                    update_video_description(video_id, old_link, new_link)